In [2]:
# https://agentpy.readthedocs.io/en/latest/

# Model design
import agentpy as ap
import random
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [3]:
classlist = []
class settlementModel(ap.Model):

    def setup(self):

        # Create agents (houses)
        ## "house density" and "size" are defined parameters
        ## Square the length of grid
        ## Multiply by density to obtain number of agents
        # AgentList function initializes the agents of desired number.
        n_houses = int(self.p['house density'] * (self.p.size**2))
        houses = self.agents = ap.AgentList(self, n_houses)

        # Create grid (settlement)
        ## Grid composed of length^2 spots which is empty
        self.settlement = ap.Grid(self, [self.p.size]*2, track_empty=True)
        ## Add the agents into the house at random
        self.settlement.add_agents(houses, random=True, empty=True)
        
        # Create roads
        for majroad in range(self.p['number_of_roads']):
            # Create path of road
            majroad_origin = np.random.randint(self.p.size, size=2)
            majroad_direction = np.random.rand()*np.pi
            for i in range(100):
                # Extend path in specified direction
                road = np.mod(np.int_(np.trunc([
                    majroad_origin[0] + i*np.cos(majroad_direction), 
                    majroad_origin[1] + i*np.sin(majroad_direction)])), 100)
                # Remove agents in path and their immediate neighbors
                road_grid = self.settlement.grid[road[0], road[1]]
                for agset in road_grid:
                    for ag in agset:
                        ag_neighbors = self.settlement.neighbors(ag,1).to_list()
                        self.settlement.remove_agents(ag_neighbors)
                    self.settlement.remove_agents(agset)
                # Create narrower side streets in random intervals
                if i%7 == 0 and np.random.rand() < 0.5:
                    for q in range(-25,25):
                        subroad = np.clip(np.int_(np.trunc([
                            road[0] + q*np.cos(majroad_direction + np.pi/2), 
                            road[1] + q*np.sin(majroad_direction + np.pi/2)])), 0, 99)
                        subroad_grid = self.settlement.grid[subroad[0],subroad[1]]
                        for ag2 in subroad_grid:
                            self.settlement.remove_agents(ag2)
                            
        # Remove deleted agents from self.agents
        self.agents = self.settlement.agents.to_list()

        # Initiate a dynamic variable for all houses
        # Condition 0: wooden, 1: strong, 2: Burning, 3: Burned
        self.agents.condition = 0
        self.agents.burnstate = 0
        self.agents.material = "wooden"
        
        number_strong = int(self.p['percentage_of_strong']* len(self.agents))
        strong_houses = self.agents.random(number_strong)
        strong_houses.condition = 1
        strong_houses.material = "strong"
        
        # Start a fire from the center of the grid
        ## 3 columns, all rows
        ## TODO change method of removing agents to not mess this up
        if self.p['random_initial_burning'] == True: 
            unfortunate_houses = self.agents.random(self.p.initial)
            unfortunate_houses.condition = 2
        if self.p['random_initial_burning'] == False: 
            initial_burning = self.settlement.agents[50,50]
            initial_burning.condition = 2
        
        self.firefighter_count = 0

    def step(self):
        
        embers = self.agents.select(self.agents.condition == 5)
        for house in embers:
            house.condition = 2

        # Select burning houses
        burning_houses = self.agents.select(self.agents.condition == 2)

        # Spread fire
        for house in burning_houses:
            neighbors = self.settlement.neighbors(house).to_list()
            # Spread by direct contact
            for neighbor in neighbors:
                if neighbor.condition == 0 or neighbor.condition == 1:
                    burnProbability = self.p['probability-of-spread']
                    direction = list(np.subtract(self.settlement.positions[neighbor],self.settlement.positions[house]))
                    if direction == [-1,0]:
                        burnProbability -= self.p['south-wind-speed']
                    elif direction == [1,0]:
                        burnProbability += self.p['south-wind-speed']
                    elif direction == [0,-1]:
                        burnProbability -= self.p['east-wind-speed']
                    elif direction == [0,1]:
                        burnProbability += self.p['east-wind-speed']
                    else:
                        continue
                    if neighbor.material == "wooden":
                        if np.random.rand() < burnProbability:
                            neighbor.condition = 2
                    elif neighbor.material == "strong":
                        if np.random.rand() < burnProbability/2:
                            neighbor.condition = 2
            # Spread by embers
            if self.p['big-jumps'] == True:
                ember = np.trunc([self.p['south-wind-speed']*5, self.p['east-wind-speed']*5])
                hloc = self.settlement.positions[house]
                floc = np.clip([int(hloc[0]+ember[0]), int(hloc[1]+ember[1])], 0, 99)
                farneighbor = self.settlement.grid[floc[0], floc[1]][0]
                for m in farneighbor:
                   # n = self.settlement.neighbors(m, 1)
                    #for x in n:
                    if np.random.rand() <= self.p['probability-of-spread'] and m.condition in [0,1]:
                        m.condition = 5
            
            # Burn out after n time steps burning
            house.burnstate += 1
            if house.material == "wooden":
                if house.burnstate >= 1:
                    house.condition = 3
            if house.material == "strong":
                if house.burnstate >= 3:
                    house.condition = 3
                
        # Introduce new state 4: firefighting response

        burned_houses = len(self.agents.select(self.agents.condition == 3))
        if burned_houses/len(self.agents) >= self.p['firefighter-threshold']:
            if self.firefighter_count > self.p['response_time']:
                if len(burning_houses) > 0:
                    firefighter_location = burning_houses.random(1)
                    for house in firefighter_location:
                        house.condition = 4
            else: 
                self.firefighter_count += 1
        
        if self.firefighter_count > self.p['response_time']:
            firefighting_houses = self.agents.select(self.agents.condition == 4)
            for house in firefighting_houses:
                neighbors = self.settlement.neighbors(house, 3)
                for neighbor in neighbors:
                    if neighbor.condition == 2:
                        neighbor.condition = 4


        # Stop simulation if no fire is left
        if len(burning_houses) == 0:
            # Getting the percentage of houses burned at the end of the simulation
            remaining_houses = len(self.agents.select(self.agents.condition <= 1))
            self.remaining_houses = remaining_houses/len(self.agents)
            burning = 1-self.remaining_houses
            classlist.append(burning)
            self.stop()

    def end(self):
        pass
        # Document a measure at the end of the simulation
        


In [5]:
# Define parameters
classlist = []
parameters = {
    'house density': 0.9, # Percentage of grid covered by houses
    'percentage_of_strong': 0.6,
    'initial': 1,
    'random_initial_burning': True,
    'size': 100, # Height and length of the grid
    'steps': 200,
    'probability-of-spread': 0.6,
    'south-wind-speed': 0.5,
    'east-wind-speed': 0.5,
    'firefighter-threshold': 0.1,
    'big-jumps': True,
    'response_time' : 30,
    'number_of_roads': 4
}

# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.settlement.attr_grid('condition')
    color_dict = {0:'#CDCDCD', 1: '#999999', 2:'#BF3E2D', 3:'#FFB159', 4:'#4248FF', 5: '#000000', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of fire in informal settlements\n"
                 f"Time-step: {model.t}, Houses left: "
                 f"{len(model.agents.select(model.agents.condition <= 1) + model.agents.select(model.agents.condition == 1))}")

fig, ax = plt.subplots(figsize = (8,8))
model = settlementModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

In [6]:
# Getting the mean and variance of the percentage of burned houses
# We did 30 sample tests. 
classlist = []
for i in range(30):
    model = settlementModel(parameters)
    model.run()


print(np.mean(classlist))
print(np.var(classlist))
print(len(classlist))
print(classlist)

Completed: 20 steps
Run time: 0:00:00.365333
Simulation finished
Completed: 67 steps
Run time: 0:00:00.793722
Simulation finished
Completed: 58 steps
Run time: 0:00:00.688627
Simulation finished
Completed: 80 steps
Run time: 0:00:00.748681
Simulation finished
Completed: 14 steps
Run time: 0:00:00.264240
Simulation finished
Completed: 50 steps
Run time: 0:00:00.716164
Simulation finished
Completed: 78 steps
Run time: 0:00:00.734233
Simulation finished
Completed: 68 steps
Run time: 0:00:00.755695
Simulation finished
Completed: 8 steps
Run time: 0:00:00.245472
Simulation finished
Completed: 43 steps
Run time: 0:00:00.515139
Simulation finished
Completed: 41 steps
Run time: 0:00:00.430902
Simulation finished
Completed: 2 steps
Run time: 0:00:00.415514
Simulation finished
Completed: 7 steps
Run time: 0:00:00.222201
Simulation finished
Completed: 53 steps
Run time: 0:00:00.514468
Simulation finished
Completed: 38 steps
Run time: 0:00:00.643586
Simulation finished
Completed: 6 steps
Run time: